In [ ]:
!pip install simpletransformers
import pandas as pd
from sklearn.metrics import classification_report
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

##Load Data

In [3]:
filepath="./data/"

In [4]:
train = pd.read_csv(filepath+"train.csv")
test = pd.read_csv(filepath+"test.csv")
go_emotion = pd.read_csv(filepath+"goemotion.csv")

In [7]:
basetrain = pd.DataFrame(train.Comments)
basetrain['label'] = train.label

In [8]:
neg_emotion=['gratitude','joy']
pos_emotion=['anger','disgust']
fake_emotion=['surprise']

In [9]:
emotions_positive=[]
emotions_negative=[]
emotions_fake=[]
for i in range(len(go_emotion)):
  for e in pos_emotion:
     if go_emotion.iloc[i][e]==1:
        emotions_positive.append(go_emotion.text[i])
  for e in neg_emotion:
     if go_emotion.iloc[i][e]==1:
        emotions_negative.append(go_emotion.text[i])
  for e in fake_emotion:
     if go_emotion.iloc[i][e]==1:
        emotions_fake.append(go_emotion.text[i])

In [10]:
#take away multi-labels from original file
same=[e for e in emotions_negative if e  in emotions_positive or e in emotions_fake]

In [11]:
emotions_negative_new=[e for e in emotions_negative if e not in same]
emotions_positive_new=[e for e in emotions_positive if e not in same]
emotions_fake_new=[e for e in emotions_fake if e not in same]


In [12]:
emotion_contents=emotions_positive_new[:1500]+list(emotions_negative_new[:2000])+list(emotions_fake_new[:200])
emotions_label=[1]*1500+[0]*2000+[2]*200

In [13]:
train_emotion = pd.DataFrame(emotion_contents)
train_emotion['label'] = emotions_label
train_emotion_shuffle=train_emotion.sample(frac=1)

In [16]:
def training(model,location,traindata,testdata,retraindata,EAT):

    model_base = ClassificationModel(model, location,num_labels=3, args={'num_train_epochs':4, 'train_batch_size':32, 'max_seq_length':400,'overwrite_output_dir': True,"use_multiprocessing": False,"use_multiprocessing_for_evaluation": False})
    model_base.train_model(traindata)
    if(EAT==1):
      model_base.train_model(retraindata)
      predictions, raw_outputs = model_base.predict(list(testdata.Comments))
      print("....................EAT RESULTS..................")
      print(classification_report(testdata['label'], predictions))
    else:
      predictions, raw_outputs = model_base.predict(list(testdata.Comments))
      print(classification_report(testdata['label'], predictions))



## MPNET

In [ ]:
#Baseline
basemodel=training("mpnet","microsoft/mpnet-base",basetrain,test,_,0)
print("-----------------------------End of Baseline-----------------------------------------")
#ZSC
zscmodel=training("mpnet","microsoft/mpnet-base",train_emotion_shuffle,test,_,0)
print("-----------------------------End of ZSC----------------------------------------------")
#FSC
zscmodel=training("mpnet","microsoft/mpnet-base",train_emotion_shuffle,test,basetrain,1)
print("-----------------------------End of FSC----------------------------------------------")

##Electra

In [ ]:
##Baseline
basemodel=training("electra","google/electra-small-discriminator",basetrain,test,_,0)
print("-----------------------------End of Baseline-----------------------------------------")

#ZSC
zscmodel=training("electra","google/electra-small-discriminator",train_emotion_shuffle,test,_,0)
print("-----------------------------End of ZSC----------------------------------------------")

#FSC
zscmodel=training("electra","google/electra-small-discriminator",train_emotion_shuffle,test,basetrain,1)
print("-----------------------------End of FSC----------------------------------------------")

##Roberta

In [ ]:
##Baseline
basemodel=training("roberta","roberta-base",basetrain,test,_,0)
print("-----------------------------End of Baseline-----------------------------------------")

#ZSC
zscmodel=training("roberta","roberta-base",train_emotion_shuffle,test,_,0)
print("-----------------------------End of ZSC----------------------------------------------")

#FSC
zscmodel=training("roberta","roberta-base",train_emotion_shuffle,test,basetrain,1)
print("-----------------------------End of FSC----------------------------------------------")

##XLnet

In [ ]:
##Baseline
basemodel=training('xlnet','xlnet-base-cased',basetrain,test,_,0)
print("-----------------------------End of Baseline-----------------------------------------")

#ZSC
zscmodel=training('xlnet','xlnet-base-cased',train_emotion_shuffle,test,_,0)
print("-----------------------------End of ZSC----------------------------------------------")

#FSC
zscmodel=training('xlnet','xlnet-base-cased',train_emotion_shuffle,test,basetrain,1)
print("-----------------------------End of FSC----------------------------------------------")

##BERT

In [ ]:
##Baseline
basemodel=training("bert","bert-base-cased",basetrain,test,_,0)
print("-----------------------------End of Baseline-----------------------------------------")

#ZSC
zscmodel=training("bert","bert-base-cased",train_emotion_shuffle,test,_,0)
print("-----------------------------End of ZSC----------------------------------------------")

#FSC
zscmodel=training("bert","bert-base-cased",train_emotion_shuffle,test,basetrain,1)
print("-----------------------------End of FSC----------------------------------------------")

##Distilbert

In [ ]:
##Baseline
basemodel=training("distilbert","distilbert-base-uncased",basetrain,test,_,0)
print("-----------------------------End of Baseline-----------------------------------------")

#ZSC
zscmodel=training("distilbert","distilbert-base-uncased",train_emotion_shuffle,test,_,0)
print("-----------------------------End of ZSC----------------------------------------------")

#FSC
zscmodel=training("distilbert","distilbert-base-uncased",train_emotion_shuffle,test,basetrain,1)
print("-----------------------------End of FSC----------------------------------------------")

##T5

In [ ]:
from simpletransformers.t5 import T5Model, T5Args
model_args = T5Args()
model = T5Model("t5","t5-small",args={'num_train_epochs':2, 'overwrite_output_dir': True})


### Prepare data for T5

In [21]:
test_df_t5=pd.DataFrame({'prefix':(["multi classification"]*len(test))})
train_df_t5=pd.DataFrame({'prefix':(["multi classification"]*len(basetrain))})
test_df_t5['input_text']=list(test.Comments)
train_df_t5['input_text']=list(train.Comments)

In [25]:
labelconvert=[]
for i in test['label']:
  if i==0:labelconvert.append('no-cyberbullying')
  if i==1:labelconvert.append('harassment')
  if i==2:labelconvert.append('defamation')

test_df_t5['target_text']=labelconvert

In [26]:
labelconvert=[]
for i in train['label']:
  if i==0:labelconvert.append('no-cyberbullying')
  if i==1:labelconvert.append('harassment')
  if i==2:labelconvert.append('defamation')

train_df_t5['target_text']=labelconvert

In [28]:
train_emotion_t5=pd.DataFrame({'prefix':(["multi classification"]*len(emotion_contents))})
train_emotion_t5['input_text']=emotion_contents
labelconvert=[]
for i in emotions_label:
  if i==0:labelconvert.append('no-cyberbullying')
  if i==1:labelconvert.append('harassment')
  if i==2:labelconvert.append('defamation')

train_emotion_t5['target_text']=labelconvert

###Baseline

In [ ]:
model = T5Model("t5","t5-small",args={'num_train_epochs':2, 'overwrite_output_dir': True})
model.train_model(train_df_t5)
predictions= model.predict([c for c in test.Comments])
labels=[]
for p in predictions:
  if 'no-cyberbullying' in p.split():
    labels.append(0)

  elif 'harassment' in p.split():
    labels.append(1)

  elif 'defamation' in p.split():
    labels.append(2)

  else:labels.append(0)
print(classification_report(test['label'], labels))

###ZSC

In [ ]:
model = T5Model("t5","t5-small",args={'num_train_epochs':2, 'overwrite_output_dir': True})
model.train_model(train_emotion_t5)
predictions= model.predict([c for c in test.Comments])
labels=[]
for p in predictions:
  if 'no-cyberbullying' in p.split():
    labels.append(0)

  elif 'harassment' in p.split():
    labels.append(1)

  elif 'defamation' in p.split():
    labels.append(2)

  else:labels.append(0)
print(classification_report(test['label'], labels))

###FSC

In [ ]:
model = T5Model("t5","t5-small",args={'num_train_epochs':2, 'overwrite_output_dir': True})
model.train_model(train_df_t5)
predictions= model.predict([c for c in test.Comments])
labels=[]
for p in predictions:
  if 'no-cyberbullying' in p.split():
    labels.append(0)

  elif 'harassment' in p.split():
    labels.append(1)

  elif 'defamation' in p.split():
    labels.append(2)

  else:labels.append(0)
print(classification_report(test['label'], labels))